# Suspect svchost Activity
It is extremely abnormal for svchost.exe to spawn without any CLI arguments and is normally observed when a malicious process spawns the process and injects code into the process memory space.

## Rule Content
```
- title: Suspect svchost Activity
  id: 16c37b52-b141-42a5-a3ea-bbe098444397
  status: experimental
  description: It is extremely abnormal for svchost.exe to spawn without any CLI arguments
    and is normally observed when a malicious process spawns the process and injects
    code into the process memory space.
  references:
  - https://securitybytes.io/blue-team-fundamentals-part-two-windows-processes-759fe15965e2
  author: David Burkett
  date: 12/28/2019
  tags:
  - attack.t1055
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection1:
      CommandLine: null
    selection2:
      Image: '*\svchost.exe'
    filter:
      ParentImage:
      - '*\rpcnet.exe'
      - '*\rpcnetp.exe'
    condition: (selection1 and selection2) and not filter
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - rpcnet.exe / rpcnetp.exe which is a lojack style software. https://www.blackhat.com/docs/us-14/materials/us-14-Kamlyuk-Kamluk-Computrace-Backdoor-Revisited.pdf
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((NOT _exists_:process_command_line AND process_path.keyword:*\\svchost.exe) AND (NOT (process_parent_path.keyword:(*\\rpcnet.exe OR *\\rpcnetp.exe))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()